## Perform enrichment test using phenotype ontology

Given a file of genes:

```
$ head data/rp-genes.tsv
NCBIGene:6295   SAG
NCBIGene:1258   CNGB1
NCBIGene:3614   IMPDH1
NCBIGene:26121  PRPF31
```

The example file here is derived from a Monarch query for all retinitis pigmentosa genes.

We want to test each class in HPO to see if it is enriched for genes in this set.

First we need to parse the gene Ids in the file

In [18]:
## Parse ids from file
file = open("data/rp-genes.tsv", "r")
gene_ids = [row.split("\t")[0] for row in file]

## show first 10 IDs:
gene_ids[:10]

['NCBIGene:6295',
 'NCBIGene:1258',
 'NCBIGene:3614',
 'NCBIGene:26121',
 'NCBIGene:7275',
 'NCBIGene:55857',
 'NCBIGene:79797',
 'NCBIGene:10594',
 'NCBIGene:64218',
 'NCBIGene:7401']

In [8]:
## Create an ontology factory in order to fetch HPO
from ontobio.ontol_factory import OntologyFactory

ofactory = OntologyFactory()
ont = ofactory.create("hp")  ## Load HP. Note the first time this runs Jupyter will show '*' - be patient

In [9]:
## Create an association factory to get gene-phenotype associations
from ontobio.assoc_factory import AssociationSetFactory
afactory = AssociationSetFactory()
## Load Associations from Monarch. Note the first time this runs Jupyter will show '*' - be patient
aset = afactory.create(ontology=ont, subject_category='gene', object_category='phenotype', taxon='NCBITaxon:9606')


In [12]:
## Run enrichment tests using all classes in ontology
enr = aset.enrichment_test(subjects=gene_ids, threshold=0.00005, labels=True)

In [19]:
## Show first 20 results
for r in enr[:20]:
        print("{:8.3g} {} {:40s}".format(r['p'],r['c'],str(r['n'])))

1.24e-123 HP:0030466 Abnormal full-field electroretinogram   
4.25e-111 HP:0000512 Abnormal electroretinogram              
6.94e-104 HP:0000546 Retinal degeneration                    
1.28e-103 HP:0030453 Abnormal visual electrophysiology       
3.03e-102 HP:0008323 Abnormal light- and dark-adapted electroretinogram
7.84e-97 HP:0000608 Macular degeneration                    
1.08e-95 HP:0001103 Abnormality of the macula               
1.66e-93 HP:0000556 Retinal dystrophy                       
3.85e-90 HP:0001147 Retinal exudate                         
5.53e-90 HP:0000662 Nyctalopia                              
4.51e-89 HP:0007675 Progressive night blindness             
1.37e-88 HP:0008020 Progressive cone degeneration           
1.55e-85 HP:0007703 Abnormality of retinal pigmentation     
2.43e-84 HP:0030506 Yellow/white lesions of the retina      
 9.5e-84 HP:0200065 Chorioretinal degeneration              
4.22e-83 HP:0001730 Progressive hearing impairment          
2.91e-81 

Given that the initial gene set is for retinitis pigmentosa genes, it's not surprising that enriched phenotype
terms are related to retinal degeneration

## Viewing Results

We can use different visualization options to see the enriched terms. First we will show a simple tree view


In [14]:
## Get all enriched class Ids
terms = [r['c'] for r in enr]

In [15]:
## Create a minimal slim of HPO consisting of enriched terms,
## with non-informative intermediate nodes removed
from ontobio.slimmer import get_minimal_subgraph
g = get_minimal_subgraph(ont.get_graph(), terms)

In [17]:
## Render as ascii-tree
from ontobio.graph_io import GraphRenderer
w = GraphRenderer.create('tree')
w.write(g, query_ids=terms)

. HP:0000118 ! Phenotypic abnormality
 % HP:0000478 ! Abnormality of the eye * 
  % HP:0012372 ! Abnormal eye morphology * 
   % HP:0012374 ! Abnormality of the globe * 
    % HP:0100887 ! Abnormality of globe size * 
     % HP:0000568 ! Microphthalmia * 
    % HP:0008056 ! Aplasia/Hypoplasia affecting the eye * 
     % HP:0000568 ! Microphthalmia * 
     % HP:0008057 ! Aplasia/Hypoplasia affecting the fundus * 
      % HP:0008061 ! Aplasia/Hypoplasia of the retina * 
       % HP:0007770 ! Hypoplasia of the retina * 
    % HP:0008047 ! Abnormality of the vasculature of the eye * 
     % HP:0008046 ! Abnormality of the retinal vasculature * 
      % HP:0007843 ! Attenuation of retinal blood vessels * 
    % HP:0004329 ! Abnormality of the posterior segment of the globe * 
     % HP:0001098 ! Abnormality of the fundus * 
      % HP:0000479 ! Abnormality of the retina * 
       % HP:0000532 ! Chorioretinal abnormality * 
        % HP:0200065 ! Chorioretinal degeneration * 
         % HP:0

## visualization

Now we will show enriched terms in a graph using graphviz

In [22]:
terms = [r['c'] for r in enr[:30]]
g = get_minimal_subgraph(ont.get_graph(), terms)
w = GraphRenderer.create('png')
w.outfile = "output/enr.png"
w.write(g, query_ids=terms)

## Image

![title](output/enr.png)